In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install opensmile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 996.2/996.2 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.4/138.4 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.4/167.4 kB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iso-639: filename=iso_639-0.4.5-py3-none-any.whl size=168840 sha256=b1314ddb0ef5fde3fe7b29b334acc246e6512678dbd193ab10e81315ca3d831f
  Stored in directory: /root/.cache/pip/wheels/d8/78/cc/5478ca3b1c3f602eae6f8cdbd78f909c0a0bfa0bbcb5c7771f
Successfully built iso-639


In [ ]:
fcsv = "/content/drive/MyDrive/MayoProject1VoxCeleb/voxceleb_age_gender_metadata_wav.csv"

metadata = {}
#uid: participant id, age, gender

import csv
reader = csv.DictReader(open(fcsv))
for row in reader:
    uid = row['uid']
    pid = row['participantid']
    age = row['age']
    gen = row['gender']
    metadata[uid] = {'age': age, 'gender': gen, 'participant': pid}
print(metadata)

{'3ca0d28e-3d07-424f-8227-3b1a3af1396e': {'age': '37.0', 'gender': 'm', 'participant': 'id00012'}, '9f7a4e7b-9b5d-4472-98bc-0a41730a4e64': {'age': '37.0', 'gender': 'm', 'participant': 'id00012'}, '5bae7482-ed31-4739-bde7-0f1f750f19b9': {'age': '37.0', 'gender': 'm', 'participant': 'id00012'}, 'b4498761-9d86-4544-b8cf-a07a3cb435ab': {'age': '37.0', 'gender': 'm', 'participant': 'id00012'}, 'c073db51-0b67-4937-98a9-b7ec73bbec75': {'age': '37.0', 'gender': 'm', 'participant': 'id00012'}, '7ed09855-1f07-4c65-b427-57a6059ef127': {'age': '37.0', 'gender': 'm', 'participant': 'id00012'}, '2b878026-1601-48c4-8478-b831d0b2eb89': {'age': '23.0', 'gender': 'm', 'participant': 'id00016'}, '7b3395dd-8df8-450e-a4df-c9eae1dc22e9': {'age': '23.0', 'gender': 'm', 'participant': 'id00016'}, 'db2fd18f-a822-4a9a-afba-b281763d87d1': {'age': '23.0', 'gender': 'm', 'participant': 'id00016'}, '098d316f-afd4-471f-a016-5f260ce4836b': {'age': '29.0', 'gender': 'm', 'participant': 'id00017'}, '60b28755-8531-4354

In [ ]:
##Opensmile Feature Extraction
import pandas as pd
import opensmile
import csv
import librosa
from math import floor
import time

from sklearn.svm import SVC
import pickle
import numpy as np
import os
import sys
import pandas as pd
from pathlib import Path

In [ ]:
#function to extract audio features from audio file
def OpenSmileAnalysis(file):
    global period
    period = 2 #2 second windows
    store = []
    signal,sr = librosa.load(file, sr=None)
    #duration = librosa.get_duration(filename = file, sr=sr) #obtains duration of audio file in seconds
    duration = librosa.get_duration(filename=file, sr=sr)
    periods = floor(duration/period) #find number of segments within audio file (in periods)
    signal = librosa.to_mono(signal)
    for i in range(periods):
        store.append(SegmentAnalysis(signal,sr,i))
    return store

#function to extract audio features from slice within audio file
def SegmentAnalysis(signal,sr,i):
    GMAPsmile = opensmile.Smile(
        feature_set=opensmile.FeatureSet.eGeMAPSv02,
        feature_level=opensmile.FeatureLevel.Functionals)
    y = GMAPsmile.process_signal(signal,sampling_rate=sr,start=pd.Timedelta(i*1000000000*period),end=pd.Timedelta((i+1)*1000000000*period))
    y = y.reset_index()
    return y


def main_extract(file):
    store = [] #contain extracted data
    print("Beginning audio extraction....")
    starttime = time.perf_counter()
    for item in OpenSmileAnalysis(file):
        store.append(item)
    data = store[0]
    #print(data)
    for item in store[1:]:
        data = data.append(item)
    print(f"Finished feature extraction in {time.perf_counter()-starttime:0.2f} seconds")
    #print(data.head())
    return data

In [ ]:
def preprocess(data):
  data = pd.DataFrame(data, columns=data.columns)
  return data

In [ ]:
# extarct opensmile features and also add metadata --> age, gender, stage to pandas dataframe that it will return
def extract_features(file):
  data = main_extract(file)
  DROP_COLS = ['start','end']
  data = data.drop(DROP_COLS, axis = 1)
  df_data = preprocess(data)
  l = len(str(file))
  st = str(file)
  name = st[55:l-13]
  df_data["age"] = metadata[name]["age"]
  df_data["gender"] = metadata[name]["gender"]
  df_data.insert(loc=0, column="ID", value=name)
  return df_data

In [ ]:
import warnings
warnings.filterwarnings("ignore")
directory = "/content/drive/MyDrive/MayoProject1VoxCeleb/structured"
all_folders = Path(directory).glob('*')
data_list = []
ctr = 0
for folder in all_folders:
    all_files = Path(folder).glob('*')
    for file in all_files:
      l1 = len(str(file))
      st1 = str(file)
      if st1[l1-4:l1] == ".wav":
        data = extract_features(file)
        #print(str(file) + ".......................................................................")
        #display(data)
        data_list.append(data)
        ctr += 1
        print(str(ctr) + "/7060 *************************************")

df_merged = pd.concat(data_list, ignore_index=True, sort=False)
print("DF MERGED.......................................................................")
display(df_merged)
df_merged.to_csv("train.csv", index = True)
from google.colab import files
files.download("train.csv")

Streaming output truncated to the last 5000 lines.
1861/7060 *************************************
Beginning audio extraction....
Finished feature extraction in 0.63 seconds
1862/7060 *************************************
Beginning audio extraction....
Finished feature extraction in 1.83 seconds
1863/7060 *************************************
Beginning audio extraction....
Finished feature extraction in 1.51 seconds
1864/7060 *************************************
Beginning audio extraction....
Finished feature extraction in 0.55 seconds
1865/7060 *************************************
Beginning audio extraction....
Finished feature extraction in 0.73 seconds
1866/7060 *************************************
Beginning audio extraction....
Finished feature extraction in 0.64 seconds
1867/7060 *************************************
Beginning audio extraction....
Finished feature extraction in 0.55 seconds
1868/7060 *************************************
Beginning audio extraction....
Finished 

,ID,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,F0semitoneFrom27.5Hz_sma3nz_meanFallingSlope,...,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,age,gender
0,0a01d6b7-465d-4c03-9362-9945629122b2,27.943003,0.124328,24.812248,26.861307,31.699125,6.886877,85.718102,47.585030,38.312286,...,0.047353,4.020101,3.645833,0.151429,0.079898,0.088750,0.064311,-33.909988,68.0,m
1,0a01d6b7-465d-4c03-9362-9945629122b2,26.467838,0.072899,24.903242,26.110186,28.646021,3.742779,171.762619,140.664597,16.289448,...,0.048154,6.030151,3.626943,0.197143,0.088756,0.070000,0.036968,-35.213783,68.0,m
2,0a01d6b7-465d-4c03-9362-9945629122b2,27.489487,0.048744,26.296909,27.341871,28.358173,2.061264,242.362473,179.615799,24.469116,...,0.032970,3.517588,3.125000,0.138333,0.076685,0.161667,0.229885,-34.446758,68.0,m
3,0a01d6b7-465d-4c03-9362-9945629122b2,26.608780,0.127619,24.208565,25.634075,30.679205,6.470640,91.653160,45.258709,26.399956,...,0.051809,7.035176,4.663213,0.112222,0.058267,0.082222,0.036751,-34.757946,68.0,m
4,0a01d6b7-465d-4c03-9362-9945629122b2,27.170237,0.127571,23.402758,27.096771,30.115894,6.713137,22.753971,0.000000,3.525636,...,0.062265,4.020101,2.590674,0.150000,0.104307,0.216000,0.129089,-33.650028,68.0,m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12765,d87c94cc-34e5-4752-b44f-8d2c7e475f42,29.890106,0.022680,29.231920,29.938593,30.394739,1.162819,39.143169,25.336927,9.849568,...,0.122549,4.020101,2.590674,0.292000,0.246284,0.095000,0.047170,-28.599495,31.0,f
12766,d87c94cc-34e5-4752-b44f-8d2c7e475f42,30.021532,0.088315,29.047306,29.451269,30.352144,1.304838,120.608582,113.294472,62.208469,...,0.131618,6.030151,4.639175,0.097778,0.048023,0.095556,0.071819,-29.762573,31.0,f
12767,d93b8667-95fc-470e-a467-3a5a13d87923,21.358158,0.034950,20.694593,21.378864,22.103210,1.408617,37.805107,51.083931,8.725139,...,0.113570,3.517588,3.108808,0.193333,0.079092,0.132000,0.143861,-29.687563,29.0,m
12768,d93b8667-95fc-470e-a467-3a5a13d87923,22.622227,0.267770,21.104309,21.436493,21.865860,0.761551,20.698824,14.767219,341.285889,...,0.142255,4.522613,3.608248,0.161429,0.133676,0.095714,0.079437,-28.477493,29.0,m


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# get participant id
pid = []
for ind in df_merged.index:
    id_spec = df_merged['ID'][ind]
    pid.append(metadata[id_spec]['participant'])
df_merged.insert(1, "Participant", pid, True)
display(df_merged)

,ID,Participant,F0semitoneFrom27.5Hz_sma3nz_amean,F0semitoneFrom27.5Hz_sma3nz_stddevNorm,F0semitoneFrom27.5Hz_sma3nz_percentile20.0,F0semitoneFrom27.5Hz_sma3nz_percentile50.0,F0semitoneFrom27.5Hz_sma3nz_percentile80.0,F0semitoneFrom27.5Hz_sma3nz_pctlrange0-2,F0semitoneFrom27.5Hz_sma3nz_meanRisingSlope,F0semitoneFrom27.5Hz_sma3nz_stddevRisingSlope,...,spectralFluxUV_sma3nz_amean,loudnessPeaksPerSec,VoicedSegmentsPerSec,MeanVoicedSegmentLengthSec,StddevVoicedSegmentLengthSec,MeanUnvoicedSegmentLength,StddevUnvoicedSegmentLength,equivalentSoundLevel_dBp,age,gender
0,0a01d6b7-465d-4c03-9362-9945629122b2,id01206,27.943003,0.124328,24.812248,26.861307,31.699125,6.886877,85.718102,47.585030,...,0.047353,4.020101,3.645833,0.151429,0.079898,0.088750,0.064311,-33.909988,68.0,m
1,0a01d6b7-465d-4c03-9362-9945629122b2,id01206,26.467838,0.072899,24.903242,26.110186,28.646021,3.742779,171.762619,140.664597,...,0.048154,6.030151,3.626943,0.197143,0.088756,0.070000,0.036968,-35.213783,68.0,m
2,0a01d6b7-465d-4c03-9362-9945629122b2,id01206,27.489487,0.048744,26.296909,27.341871,28.358173,2.061264,242.362473,179.615799,...,0.032970,3.517588,3.125000,0.138333,0.076685,0.161667,0.229885,-34.446758,68.0,m
3,0a01d6b7-465d-4c03-9362-9945629122b2,id01206,26.608780,0.127619,24.208565,25.634075,30.679205,6.470640,91.653160,45.258709,...,0.051809,7.035176,4.663213,0.112222,0.058267,0.082222,0.036751,-34.757946,68.0,m
4,0a01d6b7-465d-4c03-9362-9945629122b2,id01206,27.170237,0.127571,23.402758,27.096771,30.115894,6.713137,22.753971,0.000000,...,0.062265,4.020101,2.590674,0.150000,0.104307,0.216000,0.129089,-33.650028,68.0,m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12765,d87c94cc-34e5-4752-b44f-8d2c7e475f42,id08356,29.890106,0.022680,29.231920,29.938593,30.394739,1.162819,39.143169,25.336927,...,0.122549,4.020101,2.590674,0.292000,0.246284,0.095000,0.047170,-28.599495,31.0,f
12766,d87c94cc-34e5-4752-b44f-8d2c7e475f42,id08356,30.021532,0.088315,29.047306,29.451269,30.352144,1.304838,120.608582,113.294472,...,0.131618,6.030151,4.639175,0.097778,0.048023,0.095556,0.071819,-29.762573,31.0,f
12767,d93b8667-95fc-470e-a467-3a5a13d87923,id04720,21.358158,0.034950,20.694593,21.378864,22.103210,1.408617,37.805107,51.083931,...,0.113570,3.517588,3.108808,0.193333,0.079092,0.132000,0.143861,-29.687563,29.0,m
12768,d93b8667-95fc-470e-a467-3a5a13d87923,id04720,22.622227,0.267770,21.104309,21.436493,21.865860,0.761551,20.698824,14.767219,...,0.142255,4.522613,3.608248,0.161429,0.133676,0.095714,0.079437,-28.477493,29.0,m


In [ ]:
df_merged.to_csv("rawTrain_1.csv", index = True)
from google.colab import files
files.download("rawTrain_1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>